In [1]:
import submitit
from frust.stepper import Stepper
from pathlib import Path
from tooltoad.chemutils import xyz2mol

# Run single structure

In [2]:
f = Path("../structures/misc/TMPs/int3_TMP.xyz")
mols = {}
with open(f, "r") as file:
    xyz_block = file.read()
    mol = xyz2mol(xyz_block)
    mols[f.stem] = (mol, [0])

step = Stepper(list(mols.keys()), step_type="mol", save_output_dir=False)
df0 = step.build_initial_df(mols)

2025-10-02 13:56:20 INFO  frust.stepper: Working dir: .


In [3]:
def run_orca_calc(
    df,
    results_dir,
    debug=False,
    n_cores=10,
    mem_gb=30,
):
    from pathlib import Path
    name = df["custom_name"].iloc[0]

    results_dir = Path(results_dir)
    results_dir.mkdir(exist_ok=True)

    step = Stepper([name],
                step_type="none",
                debug=debug,
                save_output_dir=False,
                output_base=str(results_dir),
                n_cores=n_cores,
                memory_gb=mem_gb)

    df1 = step.orca(df, "DFT-preopt", {
        "wB97X-D3": None,
        "6-31G**": None,
        "Opt":    None,
        "NoSym":   None,
    }, xtra_inp_str=
"""%geom
    Constraints
        {B 11 10 C}
        {B 11 19 C}
        {B 12 10 C}
        {B 12 19 C}
        {B 19 10 C}
        {B 11 12 C}
        {A 10 11 19 C}
        {A 10 12 19 C}
    end
end""")

    df2 = step.orca(df1, "DFT", {
        "wB97X-D3": None,
        "6-31G**": None,
        "TightSCF": None,
        "SlowConv": None,
        "Opt":    None,
        "Freq":    None,
        "NoSym":   None,
    })
    
    # detailed_inp = """%CPCM\nSMD TRUE\nSMDSOLVENT "chloroform"\nend"""
    # df2 = step.orca(df1, "DFT-SP", {
    #     "wB97X-D3": None,
    #     "6-31+G**": None,
    #     "TightSCF": None,
    #     "SP":       None,
    #     "NoSym":    None,
    # },detailed_inp)

    df2.to_parquet(f"{results_dir}/results_{name}.parquet")

In [4]:
DEBUG           = False
N_CORES         = 1
MEM_GB          = 20
TIMEOUT_MIN     = 14400
RESULTS_DIR     = "int3_TMP_1"

executor = submitit.AutoExecutor(f"logs/{RESULTS_DIR}")
executor.update_parameters(
    slurm_partition="kemi1",
    cpus_per_task=N_CORES,
    mem_gb=MEM_GB,
    timeout_min=TIMEOUT_MIN,
)

name = df0["custom_name"].iloc[0]
executor.update_parameters(slurm_job_name=name)
executor.submit(run_orca_calc, df0, RESULTS_DIR, DEBUG, N_CORES, MEM_GB)
print(f"Submitted: {name}")

Submitted: int3_TMP
